## Bibliotecas e carregamento de arquivo

In [1]:
#imports da vida
import pandas as pd
import numpy as np
#import re
import requests
#import os
from bs4 import BeautifulSoup
import unidecode

#import matplotlib.pyplot as plt
#import seaborn as sns
#from matplotlib.ticker import FuncFormatter
from datetime import datetime

In [2]:
#carrega os arquivos
df = pd.read_csv('02-DF.csv')

In [3]:
url_site = "https://www.transfermarkt.us/" ## https://www.transfermarkt.us/Jan-Oblak/leistungsdatendetails/spieler/121483 ## https://www.transfermarkt.us/Jan-Oblak/leistungsdatendetails/spieler/121483/saison/2014/verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1 ## https://www.transfermarkt.us/Jan-Oblak/transfers/spieler/121483/transfer_id/1087965

# Get headers from https://www.whatismybrowser.com/detect/what-http-headers-is-my-browser-sending
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36',
}

## Funções

### Dados - Home

In [4]:
# Definir a função
def get_home(nome_hifen, id):
    url_principal = url_site + str(nome_hifen) + "/leistungsdatendetails/spieler/" + str(id)
    response = requests.get(url_principal, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    print(url_principal)
   
    # Inicializar valorexs padrão
    date_of_birth = "N/A"
    place_of_birth = "N/A"
    citizenships = "N/A"
    height = "N/A"
    caps = "0"
    goals = "0"
    
    # Extrai a data de nascimento/idade
    birth_date_elem = soup.find('span', itemprop='birthDate')
    if birth_date_elem:
        date_of_birth = birth_date_elem.get_text(strip=True)

    # Extrai o local de nascimento
    birth_place_elem = soup.find('span', itemprop='birthPlace')
    if birth_place_elem:
        place_of_birth = birth_place_elem.get_text(strip=True)
    
    #citizenship_elem = soup.find('span', itemprop='nationality')
    #if citizenship_elem:
    #    citizenship = citizenship_elem.get_text(strip=True)

    # Extrai a altura
    height_elem = soup.find('span', itemprop='height')
    if height_elem:
        height = height_elem.get_text(strip=True)

    # Extrai Caps/Goals
    highlight_elems = soup.find_all('a', class_='data-header__content--highlight')
    if len(highlight_elems) > 1:
        caps = highlight_elems[0].get_text(strip=True)
        goals = highlight_elems[1].get_text(strip=True)
        
    
    #####    #####    #####    #####    #####    #####    #####     #####    PROFILE    #####    #####    #####    #####    #####    #####    #####    
    url_profile = url_site + str(nome_hifen) + "/profil/spieler/" + str(id)
    print(url_profile)
    response = requests.get(url_profile, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Localizar todas as tags <img> dentro do elemento relevante
    flags = soup.find_all('img', class_='flaggenrahmen')
    # Extrair os valores dos títulos ou textos alternativos
    nationalities = [flag['title'] for flag in flags]
    # Remover duplicatas enquanto preserva a ordem
    unique_nationalities = list(dict.fromkeys(nationalities))
    unique_nationalities.sort()
    citizenships = f"{len(unique_nationalities)}, {', '.join(unique_nationalities)}"
    #print(f'\n Cidadanias: {citizenships}')

    # Encontra o elemento que contém a informação sobre o pé
    foot_element = soup.find('span', class_='info-table__content info-table__content--regular', text='Foot:')
    #print(f'\n Foot: {foot_element}')

    if foot_element:
        # O próximo irmão (o conteúdo que está em <span class="info-table__content info-table__content--bold">)
        foot_value = foot_element.find_next_sibling('span', class_='info-table__content info-table__content--bold').text
        #print(f'Foot: {foot_value}')
    else:
        foot_value = "N/A"
        #print('NA')
    #####    #####    #####    #####    #####    #####    #####    #####    RETORNO   #####    #####    #####    #####    #####    #####    

    # Dados extraídos em uma lista
    data = [
        id,
        nome_hifen,
        foot_value,
        date_of_birth,
        place_of_birth,
        citizenships,
        height,
        caps,
        goals
        ]
    return data


### Dados - stats

In [4]:
def get_stats(nome_hifen, id, ano):
    ano = int(ano) - 1

    url_detalhada = url_site + str(nome_hifen) + "/leistungsdatendetails/spieler/" + str(id) + "/saison/" + str(ano) + "/verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1"
    response = requests.get(url_detalhada, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    data = []
    print(url_detalhada)
    
    # Encontrar a tabela
    table = soup.find('table', class_='items')

                
    if table:
        # Extrair todas as linhas da tabela
        rows = table.find_all('tr')

        # Inicializar variáveis
        escalado = jogou = ppm = gols_para = assistencias = gols_contra = entrou = saiu = amarelo = amarelo_duplo = vermelho = gols_de_penalti = minutos_por_gol = minutos = "0"

        # Identificar a última linha que contém o texto "Total"
        for row in rows:
            if 'Total' in row.text:
                cols = row.find_all('td')
                if len(cols) > 1:  # Verifique se a linha não está vazia
                    escalado = cols[3].get_text(strip=True)
                    jogou = cols[4].get_text(strip=True)
                    ppm = cols[5].get_text(strip=True)
                    gols_para = cols[6].get_text(strip=True)
                    assistencias = cols[7].get_text(strip=True)
                    gols_contra = cols[8].get_text(strip=True)
                    entrou = cols[9].get_text(strip=True)
                    saiu = cols[10].get_text(strip=True)
                    amarelo = cols[11].get_text(strip=True)
                    amarelo_duplo = cols[12].get_text(strip=True)
                    vermelho = cols[13].get_text(strip=True)
                    gols_de_penalti = cols[14].get_text(strip=True)
                    minutos_por_gol = cols[15].get_text(strip=True)
                    minutos = cols[16].get_text(strip=True)
    else:
        # Definir todas as variáveis como "0" se a tabela não for encontrada
        escalado = jogou = ppm = gols_para = assistencias = gols_contra = entrou = saiu = amarelo = amarelo_duplo = vermelho = gols_de_penalti = minutos_por_gol = minutos = "0"
                
    # Dados extraídos em uma lista
    data = [
        id,
        ano,
        escalado,
        jogou,
        ppm, 
        gols_para,
        assistencias,
        gols_contra,
        entrou,
        saiu,
        amarelo,
        amarelo_duplo,
        vermelho,
        gols_de_penalti,
        minutos_por_gol,
        minutos
    ]
    return data    

## preparar a base para as funções

In [5]:
# Obtém a hora atual
hora_atual = datetime.now()
hora_formatada = hora_atual.strftime("%H:%M:%S")
print("Hora Inicial:", hora_formatada)

data = []

print("AGORA VEM O LOOP")

# Loop através de cada linha do DataFrame df
for index, row in df.iterrows():
    nome_val = row['Nome'].strip().replace(' ', '-')
    nome_val = nome_val.replace('\'', '')
    nome_val = nome_val.replace('.', '')
    nome_val = unidecode.unidecode(nome_val)
    id_val = row['ID']
    ano = row['Ano da Tranferência']
    
    data.append({'nome': nome_val, 'id': id_val, 'ano': ano})
df_base = pd.DataFrame(data)
df_base = df_base.drop_duplicates()
#print(df_base)

# Obtém a hora atual
hora_atual = datetime.now()
hora_formatada = hora_atual.strftime("%H:%M:%S")
print("Hora Final:", hora_formatada)


Hora Inicial: 20:51:09
AGORA VEM O LOOP
Hora Final: 20:51:09


In [6]:
df_nome_id = df_base[['nome','id']]
df_nome_id= df_nome_id.drop_duplicates()
df_nome_id

,nome,id
0,David-Luiz,46741
1,Filipe-Luis,21725
2,Gabriel-Paulista,149498
3,Guilherme-Siqueira,44373
4,Wallace,225335
...,...,...
2057,Venaque,648022
2064,Heitor,815672
2065,Lucas-Dias,642014
2066,Vialle,880458


## Montar home.csv

In [8]:
home_columns = ["ID", "nome_hifen", "pe", "data_nasc", "local_nasc", "cidadania", "altura", "convocacoes", "gols_selecao"]
df_home = pd.DataFrame(columns=home_columns)

for index, row in df_nome_id.iterrows():
    nome = row['nome']
    id = row['id']
    print(f'\nDados de {nome}')
    lista_home=get_home(nome, id)
    df_home.loc[len(df_home)] = lista_home

df_home=df_home.drop_duplicates()
df_home.to_csv('03-DF-home.csv', index=True)


Dados de David-Luiz
https://www.transfermarkt.us/David-Luiz/leistungsdatendetails/spieler/46741
https://www.transfermarkt.us/David-Luiz/profil/spieler/46741

Dados de Filipe-Luis
https://www.transfermarkt.us/Filipe-Luis/leistungsdatendetails/spieler/21725
https://www.transfermarkt.us/Filipe-Luis/profil/spieler/21725

Dados de Gabriel-Paulista
https://www.transfermarkt.us/Gabriel-Paulista/leistungsdatendetails/spieler/149498
https://www.transfermarkt.us/Gabriel-Paulista/profil/spieler/149498

Dados de Guilherme-Siqueira
https://www.transfermarkt.us/Guilherme-Siqueira/leistungsdatendetails/spieler/44373
https://www.transfermarkt.us/Guilherme-Siqueira/profil/spieler/44373

Dados de Wallace
https://www.transfermarkt.us/Wallace/leistungsdatendetails/spieler/225335
https://www.transfermarkt.us/Wallace/profil/spieler/225335

Dados de Matheus-Doria
https://www.transfermarkt.us/Matheus-Doria/leistungsdatendetails/spieler/213841
https://www.transfermarkt.us/Matheus-Doria/profil/spieler/213841



## Montar Stats.csv

In [7]:
# Obtém a hora atual
hora_atual = datetime.now()
hora_formatada = hora_atual.strftime("%H:%M:%S")
print("Hora Inicial:", hora_formatada)

# Definir os nomes das colunas
stats_columns = ["ID", "ano_stat", "escalado", "jogou", "ppm", "gols_para", "assistencias", "gols_contra", "entrou", "saiu", "amarelo", "amarelo_duplo", "vermelho", "Gols_de_penalti", "minutos_por_gol", "minutos"]
df_stats = pd.DataFrame(columns=stats_columns)

for index, row in df_base.iterrows():
    nome = row['nome']
    id = row['id']
    ano = row['ano']
    print(f'\n{nome}')
    lista_stats=get_stats(nome, id, ano)
    df_stats.loc[len(df_stats)] = lista_stats

df_stats=df_stats.drop_duplicates()
df_stats.to_csv('03-DF-stats.csv', index=True)

# Obtém a hora atual
hora_atual = datetime.now()
hora_formatada = hora_atual.strftime("%H:%M:%S")
print("Hora Final:", hora_formatada)

Hora Inicial: 20:51:19

David-Luiz
https://www.transfermarkt.us/David-Luiz/leistungsdatendetails/spieler/46741/saison/2013/verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1

Filipe-Luis
https://www.transfermarkt.us/Filipe-Luis/leistungsdatendetails/spieler/21725/saison/2013/verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1

Gabriel-Paulista
https://www.transfermarkt.us/Gabriel-Paulista/leistungsdatendetails/spieler/149498/saison/2013/verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1

Guilherme-Siqueira
https://www.transfermarkt.us/Guilherme-Siqueira/leistungsdatendetails/spieler/44373/saison/2013/verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1

Wallace
https://www.transfermarkt.us/Wallace/leistungsdatendetails/spieler/225335/saison/2013/verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1

Matheus-Doria
https://www.transfermarkt.us/Matheus-Doria/leistungsdatendetails/spieler/213841/saison/2013/verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1

Wendell
https://www